# Entendendo o método PCA
> SVD Compression
- toc: true 
- badges: true
- comments: true
- author: Victor Nascimento
- categories: [algebra linear]
- image: images/chart-preview.png




Infelizmente o site está com problemas para renderizar figuras 3D. Para acessar o post clique no link abaixo por favor.
https://nbviewer.jupyter.org/github/VictorNascimento2000/praticandoml/blob/master/_notebooks/2020-11-16-PCA.ipynb